In [1]:
## testing PER_beahvior

## rewriting scipy peaks with a library


# import stuff
import peaks #my library

import numpy as np
from matplotlib import pyplot as plt
from pprint import pprint
import os
os.listdir(os.getcwd())
import csv as csv
import json as json
import scipy as scipy
from scipy.signal import find_peaks
import h5py


##### get paths ##########
date = '20230407'
fly_id_date_code = 'b-0407'

Path = 'G:/bruker vid 2023/20230407/results/'  #path to results files from read ROIs or DLC
SavePath = Path + 'h5_files/'
peaks.make_dirs(SavePath)


####   mostly static variables  ########
data_reducer = 100
voltage_framerate = 10000/data_reducer #frames/s # 1frame/.1ms * 1000ms/1s = 10000f/s
#with reducer i get 1 frame for every .1 * 100 ms => frame/.1*100 ms * 1000ms/s = 100f/s
#each "frame" is 0.1ms
video_framerate = 200 #f/s
#video_framerate_downstairs = 33

In [ ]:





#

files = os.listdir(Path)
voltage_files = peaks.check_for_voltage_file(Path)
roi_files = peaks.check_for_results_file(Path)

##look at voltage and roi files and add data to h5 file based on fly#
if roi_files is not None:
    for roi_file in roi_files:
        fly_number = peaks.find_fly_number(roi_file)
        h5_filename = f'{date}_fly_number_{fly_number}_.h5'
        h5_Path = os.path.join(SavePath, h5_filename)
        data = peaks.import_roi_results(os.path.join(Path, roi_file))
        peaks.add_to_h5(h5_Path, 'roi data', data)
        if peaks.check_for_key(h5_Path, 'fly-id') == False:
            fly_id = f'{fly_id_date_code}-{fly_number}'
            peaks.add_to_h5(h5_Path, 'fly-id', fly_id)
        

if voltage_files is not None:
    for voltage_file in voltage_files:
        fly_number = peaks.find_fly_number(roi_file)
        h5_filename = f'{date}_fly_number_{fly_number}_.h5'
        h5_Path = os.path.join(SavePath, h5_filename)
        data = peaks.import_voltage_data(os.path.join(Path, roi_file))
        peaks.add_to_h5(h5_Path, 'voltage data', data)
        if peaks.check_for_key(h5_Path, 'fly-id') == False:
            fly_id = f'{fly_id_date_code}-{fly_number}'
            peaks.add_to_h5(h5_Path, 'fly-id', fly_id)


#set framerate
 # **!!  change so code just asks is voltage framerate or video framerate needed


#set interval time
##since most of my experiments are switch do I need this?
interval_time = None ##decide if I should change this later

#set fly numbers 
# #(should be set when import data)


#get peaks for light and PER
##look at every flies h5 file and open and run peaks 
#    (note: some light data may be in roi file)
#  if there is more than one mean in the data then it will return a list of list of peaks for each mean





##make plots








#smooth PER peaks 






#find onset of PER peaks



#find indiv






#

In [2]:
h5files = [file for file in os.listdir(SavePath) if '.h5' in file]

for fly in h5files:
    each_path = os.path.join(SavePath, fly)
    with h5py.File(each_path, 'a') as f:
        print(f.keys())
        print(f['fly-id'][()])
        if 'roi data' in f.keys():
            roi_data = f['roi data'][()]
            print(roi_data[0:10])
            print(roi_data[0][0])
            if 'Mean' in str(roi_data[0][0]):
                print(True)
            #print(peaks.is_column_mean(roi_data, 0, 0))
            print(np.shape(roi_data))
            data_peaks, properties, columns = peaks.get_peaks(roi_data, each_path)
        if 'voltage data' in f.keys():
            voltage_data = f['voltage data'][()]
            light_peaks, light_properties, columns = peaks.get_peaks(roi_data, each_path)

<KeysViewHDF5 ['fly-id', 'light peaks', 'roi data']>
b'b-0407-1'
[[b'Mean(light_fly1)']
 [b'8.560829241680306']
 [b'8.513229678123295']
 [b'8.557828696126569']
 [b'8.568876159301691']
 [b'8.540916530278233']
 [b'8.553464266230224']
 [b'8.576377523186034']
 [b'8.555100927441353']
 [b'8.550463720676486']]
b'Mean(light_fly1)'
True
(15728, 1)
titles are [array([b'Mean(light_fly1)'], dtype='|S16')]
light
<KeysViewHDF5 ['PER peaks', 'fly-id', 'light peaks', 'roi data']>
b'b-0407-2'
[[b'Mean(light_fly2)']
 [b'6.312654320987654']
 [b'6.387962962962963']
 [b'6.3012345679012345']
 [b'6.275']
 [b'6.37716049382716']
 [b'6.25216049382716']
 [b'6.348456790123457']
 [b'6.2709876543209875']
 [b'6.421604938271605']]
b'Mean(light_fly2)'
True
(136682, 1)
titles are [array([b'Mean(light_fly2)'], dtype='|S16')]
light
<KeysViewHDF5 ['PER peaks', 'fly-id', 'light peaks', 'roi data']>
b'b-0407-3'
[[b'Mean(light_fly3)']
 [b'6.063044235458029']
 [b'6.14367816091954']
 [b'6.1706722396377565']
 [b'6.1206896551724